### Load libs, init clients and define variables

In [ ]:
from google.cloud import bigquery
from google.auth import default
from google.colab import auth
import gspread
import pandas as pd
import concurrent.futures

Init BigQuery Client with Google Auth

In [ ]:
auth.authenticate_user()
creds, _ = default()
gs_client = gspread.authorize(creds)
bq_client = bigquery.Client(credentials=creds, project='newagetest') # select newage test project ID

Define dates to parse

In [ ]:
# 2017-07-31 -> 20170731 ...
dates_to_parse = ['20170731', '20170730', '20170729', '20170728', '20170727']

### Fetching function - get DB table with given specific date



In [ ]:
def get_table(table_date):
    sql = f'''
        SELECT *
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_{table_date}`
        LIMIT 1000
        '''
    table_df = bq_client.query(sql).to_dataframe()
    return table_df

### Concurent df extraction execution

In [ ]:
# CONCURENT SOLUTION ~ 3.5s
%%timeit -n 1 -r 1

with concurrent.futures.ThreadPoolExecutor() as executor:
    dataframes = list(executor.map(get_table, dates_to_parse))

3.82 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Single thread solution for perfomance comparison

In [ ]:
# SINGLE THREAD SOLUTION ~ 11.5s
%%timeit -n 1 -r 1

for date in dates_to_parse:
    get_table(date)

11.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Concatanate DFs

In [ ]:
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.head(5)

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1501208316,1501208316,20170727,"{'visits': 1, 'hits': 16, 'pageviews': 15, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 19, 'minu...",5363034362387281051,None,None,Organic Search,Not Socially Engaged
1,<NA>,1,1501215762,1501215762,20170727,"{'visits': 1, 'hits': 16, 'pageviews': 14, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 21, 'minu...",4351689792849469595,None,None,Organic Search,Not Socially Engaged
2,<NA>,89,1501174589,1501174589,20170727,"{'visits': 1, 'hits': 16, 'pageviews': 11, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 9, 'minut...",3937673380007666721,None,None,Organic Search,Not Socially Engaged
3,<NA>,4,1501204999,1501204999,20170727,"{'visits': 1, 'hits': 16, 'pageviews': 11, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 18, 'minu...",5546793669751235607,None,None,Referral,Not Socially Engaged
4,<NA>,9,1501205850,1501205850,20170727,"{'visits': 1, 'hits': 16, 'pageviews': 16, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 18, 'minu...",6014980722805375499,None,None,Referral,Not Socially Engaged


### Grouping and Aggregating Data

In [ ]:
# Expand totals, device and trafficSource columns
totals_df = pd.json_normalize(combined_df['totals'])
device_df = pd.json_normalize(combined_df['device'])
traffic_df = pd.json_normalize(combined_df['trafficSource'])

# add expanded features
combined_df = pd.concat([combined_df, totals_df, device_df, traffic_df], axis=1)

Create daily, campaign and device insights

In [ ]:
daily_insights = combined_df.groupby('date').agg({
    'visitId': 'count',      # Кількість візитів
    'transactions': 'sum',   # Кількість транзакцій
    'transactionRevenue': 'sum',  # Сума доходів від транзакцій
    'pageviews': 'sum',      # Кількість переглядів сторінок
    'sessionQualityDim': 'mean'  # Середня якість сесій
}).reset_index()

daily_insights

,date,visitId,transactions,transactionRevenue,pageviews,sessionQualityDim
0,20170727,1000,52.0,4.348380e+09,7573.0,10.719
1,20170728,1000,0.0,0.000000e+00,1000.0,1.011
2,20170729,1000,15.0,9.028400e+08,3060.0,3.705
3,20170730,1000,0.0,0.000000e+00,1094.0,1.009
4,20170731,1000,0.0,0.000000e+00,999.0,1.009


In [ ]:
campaign_insights = combined_df.groupby('campaign').agg({
    'visitId': 'count',      # Кількість візитів
    'transactions': 'sum',   # Кількість транзакцій
    'transactionRevenue': 'sum',  # Сума доходів від транзакцій
    'pageviews': 'sum',      # Кількість переглядів сторінок
    'sessionQualityDim': 'mean'  # Середня якість сесій
}).reset_index()

campaign_insights

,campaign,visitId,transactions,transactionRevenue,pageviews,sessionQualityDim
0,(not set),4773,65.0,5.128490e+09,13044.0,3.491515
1,AW - Accessories,4,0.0,0.000000e+00,5.0,1.000000
2,AW - Dynamic Search Ads Whole Site,103,2.0,1.227300e+08,496.0,6.310680
3,Data Share Promo,120,0.0,0.000000e+00,181.0,1.116667


In [ ]:
device_insights = combined_df.groupby('deviceCategory').agg({
    'visitId': 'count',      # Кількість візитів
    'transactions': 'sum',   # Кількість транзакцій
    'transactionRevenue': 'sum',  # Сума доходів від транзакцій
    'pageviews': 'sum',      # Кількість переглядів сторінок
    'sessionQualityDim': 'mean'  # Середня якість сесій
}).reset_index()

device_insights

,deviceCategory,visitId,transactions,transactionRevenue,pageviews,sessionQualityDim
0,desktop,3181,61.0,5.095760e+09,9773.0,4.481295
1,mobile,1588,4.0,1.166900e+08,3368.0,1.688287
2,tablet,231,2.0,3.877000e+07,585.0,2.238095


### Load Aggregated data to Google Sheets

In [ ]:
# Create Google Sheets
spreadsheet = gs_client.create('BigQuery Data Transfer')
sheet1 = spreadsheet.add_worksheet(title='Daily Insights', rows=len(daily_insights), cols=len(daily_insights.columns))
sheet2 = spreadsheet.add_worksheet(title='Campaing Insights', rows=len(campaign_insights), cols=len(campaign_insights.columns))
sheet3 = spreadsheet.add_worksheet(title='Device Insights', rows=len(device_insights), cols=len(device_insights.columns))

# delete default sheet
default_sheet = spreadsheet.get_worksheet(0)
spreadsheet.del_worksheet(default_sheet)

{'spreadsheetId': '1qu2zQGcsYh1Yg5N219K13OiYrZh3LC2SLv_oYlka0VY',
 'replies': [{}]}

In [ ]:
# Function to update sheet with data
def write_to_sheet(sheet, dataframe):
    sheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

In [ ]:
# Concurent upload to Google Sheet File
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(write_to_sheet, [sheet1, sheet2, sheet3], [daily_insights, campaign_insights, device_insights])

In [ ]:
spreadsheet.url

'https://docs.google.com/spreadsheets/d/1qu2zQGcsYh1Yg5N219K13OiYrZh3LC2SLv_oYlka0VY'